In [2]:
import pandas as pd

In [24]:
from pyspark.sql import functions as F

In [8]:
spark

In [30]:
temperature_df = spark.read.option("header",True).csv("GlobalLandTemperaturesByCity.csv")
temperature_df.printSchema()

root
 |-- dt: string (nullable = true)
 |-- AverageTemperature: string (nullable = true)
 |-- AverageTemperatureUncertainty: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)



In [31]:
us_temperature_df = temperature_df.filter(F.col('Country') == 'United States')
us_temperature_df.count()

687289

In [32]:
@F.udf
def date_to_month(date_string):
    month = date_string.split("-")[1]
    return int(month)
us_temperature_df = us_temperature_df.select(
date_to_month(F.col('dt')).alias('month'),
F.lower(F.col('City')).alias('city'),
F.col('AverageTemperature').alias('avg_temperature'),
)
us_temperature_df.printSchema()

root
 |-- month: string (nullable = true)
 |-- city: string (nullable = true)
 |-- avg_temperature: string (nullable = true)



In [33]:
us_temperature_agg = us_temperature_df.groupby(['city', 'month']).agg(
F.round(F.avg('avg_temperature'),2).alias('avg_temperature'))
us_temperature_agg.show(100)t

+----------------+-----+---------------+
|            city|month|avg_temperature|
+----------------+-----+---------------+
|      cape coral|    6|          26.07|
|     gainesville|    6|           26.4|
|      louisville|   10|          13.37|
|            mesa|   12|           10.7|
|      montgomery|    1|           7.02|
|    newport news|    4|          13.24|
|         norwalk|    6|          20.53|
|    philadelphia|   11|           7.32|
|         spokane|    6|           15.5|
|      toms river|    6|          21.19|
|     west jordan|    5|          14.19|
|       anchorage|   11|         -11.06|
|       baltimore|    2|           0.93|
|         buffalo|    7|          20.89|
|  corpus christi|   12|          13.62|
|      evansville|   11|           7.59|
|      fort wayne|    9|          17.68|
|        hartford|    7|          21.57|
|      huntsville|    1|           4.37|
|         lansing|   11|           3.02|
|     little rock|    4|          16.67|
|   oklahoma cit

In [34]:
us_temperature_agg.repartition(2).write.mode('overwrite').parquet('data/temperature_transformed/')